### Import packages

In [19]:
# Chargement package
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [20]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score

In [22]:
# Connexion à ggdrive
#from google.colab import drive
#drive.mount('/content/drive')

In [23]:
# Import du fichier via ggdrive
#file_path = "/content/drive/MyDrive/Python M2/train_clean.csv" # Lien perso ggdrive
df = pd.read_csv('model_data.csv', sep = ',')

### Boosting

In [24]:
df_clean = df.copy()

In [25]:
df_clean.head()

,iid_pid,race,date,career_c,age_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,age,imprace,clubbing,concerts,music,match
0,6804,4.0,6.0,1.0,29.0,3.0,10.000000,10.000000,3.000000,10.000000,0.000000,27.0,4.0,3.0,6.0,5.0,0
1,6803,4.0,6.0,1.0,23.0,3.0,9.000000,10.000000,3.000000,10.000000,2.000000,27.0,4.0,3.0,6.0,5.0,0
2,6802,4.0,6.0,1.0,23.0,6.0,8.000000,10.000000,6.000000,9.000000,5.000000,27.0,4.0,3.0,6.0,5.0,0
3,6801,4.0,6.0,1.0,23.0,7.0,10.000000,8.000000,8.000000,8.000000,7.000000,27.0,4.0,3.0,6.0,5.0,0
4,6800,4.0,6.0,1.0,22.0,2.0,6.312235,6.594035,5.626896,4.969566,3.690192,27.0,4.0,3.0,6.0,5.0,0


In [26]:
X = df_clean[df_clean.columns[1:-1]] # On enlève la colonne ID et la colonne CLASSE
y = df_clean.match # Colonne var à prédire

In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6804 entries, 0 to 6803
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   race      6804 non-null   float64
 1   date      6804 non-null   float64
 2   career_c  6804 non-null   float64
 3   age_o     6804 non-null   float64
 4   attr_o    6804 non-null   float64
 5   sinc_o    6804 non-null   float64
 6   intel_o   6804 non-null   float64
 7   fun_o     6804 non-null   float64
 8   amb_o     6804 non-null   float64
 9   shar_o    6804 non-null   float64
 10  age       6804 non-null   float64
 11  imprace   6804 non-null   float64
 12  clubbing  6804 non-null   float64
 13  concerts  6804 non-null   float64
 14  music     6804 non-null   float64
dtypes: float64(15)
memory usage: 797.5 KB


In [28]:
print(X.shape, y.shape)

(6804, 15) (6804,)


In [29]:
y.value_counts()

0    5658
1    1146
Name: match, dtype: int64

In [30]:
# Utilisation du model_selection de sklearn
import sklearn
from sklearn import model_selection
# Affichage de la version
print(sklearn.__version__)

1.0.2


In [31]:
X_app, X_test, y_app, y_test = model_selection.train_test_split(X,y, train_size=0.7, random_state=0, stratify=y)


In [32]:
print(X_app.shape, X_test.shape, y_app.shape, y_test.shape)

(4762, 15) (2042, 15) (4762,) (2042,)


In [33]:
# Vérification de l'égalité de distribution entre y_app et y_test
print(y_app.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

0    0.831583
1    0.168417
Name: match, dtype: float64
0    0.831538
1    0.168462
Name: match, dtype: float64


In [34]:
sm = SMOTE(random_state=42, sampling_strategy=0.5, k_neighbors=5)
X_res, y_res = sm.fit_resample(X_app, y_app)

In [35]:
clf3 = GradientBoostingClassifier(n_estimators=175, learning_rate=1,max_depth=1, random_state=0,min_samples_split=15, min_samples_leaf=20).fit(X_res, y_res)
y_pred3 = clf3.predict(X_test)
print("F1 Score :",metrics.f1_score(y_test, y_pred3, average = "macro"))
print(metrics.confusion_matrix(y_test,y_pred3))

F1 Score : 0.5984912108506979
[[1599   99]
 [ 266   78]]


### Import/Prediction/Export - Submission Kaggle

In [38]:
sub = pd.read_csv('sub_clean.csv', sep = ',',index_col = [0])

In [39]:
print(sub.shape)
print(sub.head(2))

(1574, 53)
   iid_pid  gender  condtn  samerace  race_o  field_cd  race  goal  date  \
0     6805       0       2         1       4       6.0     4     2     4   
1     6806       0       2         1       4       6.0     4     2     4   

   go_out  ...  music  shopping  yoga  exphappy  attr1_1  sinc1_1  intel1_1  \
0       2  ...    7.0       7.0   7.0       6.0     10.0     20.0      30.0   
1       2  ...    7.0       7.0   7.0       6.0     10.0     20.0      30.0   

   fun1_1  amb1_1  shar1_1  
0    10.0    10.0     20.0  
1    10.0    10.0     20.0  

[2 rows x 53 columns]


In [40]:
sub_test = sub[df.columns[:-1].tolist()]
sub_test.head(2)

,iid_pid,race,date,career_c,age_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,age,imprace,clubbing,concerts,music
0,6805,4,4,16.0,25.0,7.0,10.0,10.0,10.0,10.0,8.0,28.0,2.0,4.0,5.0,7.0
1,6806,4,4,16.0,26.0,6.0,7.0,8.0,6.0,8.0,8.0,28.0,2.0,4.0,5.0,7.0


In [41]:
ysub_pred = clf3.predict(sub_test.iloc[:,1:])

In [42]:
target = pd.DataFrame(ysub_pred)
target.value_counts()

0    1457
1     117
dtype: int64

Préparation des données à l'export sur kaggle

In [43]:
sub_kaggle = pd.concat([sub.iloc[:,0].reset_index(drop=True), target], axis=1)

In [44]:
sub_kaggle.columns = ['iid_pid','target']

In [45]:
sub_kaggle.shape

(1574, 2)

In [46]:
from google.colab import files
sub_kaggle.to_csv('sub_kaggle2.csv', index = False)
files.download('sub_kaggle2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>